In [1]:
#IMPORTS

import numpy as np
import matplotlib.pyplot as plt
import pygame
import random

from pygame.locals import (
    RLEACCEL,
    K_UP,
    K_DOWN,
    K_LEFT,
    K_RIGHT,
    K_ESCAPE,
    KEYDOWN,
    QUIT,
)


pygame 2.1.2 (SDL 2.0.16, Python 3.8.16)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [69]:
#CLASS DEFINITIONS

class Drone(pygame.sprite.Sprite):
    def __init__(self, ID, xi, yi, zi, mode):
        self.x = xi
        self.y = yi
        self.z = zi
        self.ID = ID
        self.mode = mode
        super(Drone, self).__init__()
        self.surf = pygame.Surface((10, 10))
        self.surf.fill((0, 191, 255))
        self.rect = self.surf.get_rect(
            center=(xi,yi,)
        )

        self.direction = np.pi / 2
        #self.direction = random.randint(0,43) / 7
        self.force = 10

        self.l = self.rect.left
        self.t = self.rect.top
        self.prevl = self.l
        self.prevt = self.t

    def update(self,SCREEN_WIDTH, SCREEN_HEIGHT):

        self.prevl = self.l
        self.prevt = self.t


        if self.mode==0:
            self.direction=0
            self.force=0
        elif self.mode==1:
            self.direction = random.randint(0,8)
            self.force = 10
        elif self.mode==2:
            self.direction = 0
            self.force = 10

        if self.force > 0:
            angle = self.direction*np.pi/4
            dx = round(self.force * np.cos(self.direction))
            dy = -1*round(self.force * np.sin(self.direction))
            self.rect.move_ip(dx,dy)

        if self.rect.left < 0:
            self.rect.left = 0
        if self.rect.right > SCREEN_WIDTH:
            self.rect.right = SCREEN_WIDTH
        if self.rect.top <= 0:
            self.rect.top = 0
        if self.rect.bottom >= SCREEN_HEIGHT:
            self.rect.bottom = SCREEN_HEIGHT

        self.l = self.rect.left
        self.t = self.rect.top

class Drone_NSEW(pygame.sprite.Sprite):
    def __init__(self, ID, xi, yi, zi):
        self.x = xi
        self.y = yi
        self.z = zi
        self.ID = ID
        super(Drone_NSEW, self).__init__()
        self.surf = pygame.Surface((10, 10))
        self.surf.fill((0, 191, 255))
        self.rect = self.surf.get_rect(
            center=(xi,yi,)
        )

        self.move = 'r'
        #self.direction = random.randint(0,43) / 7
        self.force = 1

        self.l = self.rect.left
        self.t = self.rect.top
        self.prevl = self.l
        self.prevt = self.t

    def update(self,SCREEN_WIDTH, SCREEN_HEIGHT):

        self.prevl = self.l
        self.prevt = self.t

        if self.force > 0:
            if self.move == 'r':
                self.rect.move_ip(self.force,0)
            elif self.move =='u':
                self.rect.move_ip(0,-1*self.force)
            elif self.move == 'l':
                self.rect.move_ip(-1*self.force,0)
            elif self.move == 'd':
                self.rect.move_ip(0,self.force)

        if self.rect.left < 0:
            self.rect.left = 0
        if self.rect.right > SCREEN_WIDTH:
            self.rect.right = SCREEN_WIDTH
        if self.rect.top <= 0:
            self.rect.top = 0
        if self.rect.bottom >= SCREEN_HEIGHT:
            self.rect.bottom = SCREEN_HEIGHT

        self.l = self.rect.left
        self.t = self.rect.top

class Goal(pygame.sprite.Sprite):
    def __init__(self, x, y, z):
        super(Goal, self).__init__()
        self.surf = pygame.Surface((10, 10))
        self.surf.fill((57,255,20))
        self.rect = self.surf.get_rect(
            center=(x,y,)
        )

class Obstacle(pygame.sprite.Sprite):
    def __init__(self,x,y,z,xdim,ydim,zdim):
        super(Obstacle, self).__init__()
        self.surf = pygame.Surface((xdim,ydim))
        self.surf.fill((255,0,0))
        self.rect = self.surf.get_rect(
            center=(x,y,)
        )

class Scanner(pygame.sprite.Sprite):
    def __init__(self,left,top,xdim,ydim,dir):
        super(Scanner, self).__init__()
        self.surf = pygame.Surface((xdim,ydim))
        self.surf.fill((255,255,255))
        self.rect = self.surf.get_rect()
        self.rect.left = left
        self.rect.top = top
        self.dir = dir

    def update(self):
        if self.dir == "r":
            self.rect.move_ip(1,0)
        elif self.dir == "l":
            self.rect.move_ip(-1,0)
        elif self.dir == "u":
            self.rect.move_ip(0,-1)
        elif self.dir == "d":
            self.rect.move_ip(0,1)
        elif self.dir == "ur":
            self.rect.move_ip(1,-1)
        elif self.dir == "ul":
            self.rect.move_ip(-1,-1)
        elif self.dir == "dl":
            self.rect.move_ip(-1,1)
        elif self.dir == "dr":
            self.rect.move_ip(1,1)

        

In [40]:
#OBSERVE() DEFINITION

def observe(entity,spritesgroup,dist,scansgroup,ogroup,ggroup,dgroup):
    l=entity.rect.left
    lx=l-1
    r=entity.rect.right
    t=entity.rect.top
    tx=t-1
    b=entity.rect.bottom

    txx = t-10
    lxx = l-10


    results = np.array([[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0]])

    rscan = Scanner(r,t,1,10,"r")
    uscan = Scanner(l,tx,10,1,"u")
    lscan = Scanner(lx,t,1,10,"l")
    dscan = Scanner(l,b,10,1,"d")
    urscan = Scanner(r,txx,10,10,"ur")
    ulscan = Scanner(lxx,txx,10,10,"ul")
    dlscan = Scanner(lxx,b,10,10,"dl")
    drscan = Scanner(r,b,10,10,"dr")
    scansgroup.add(rscan,uscan,lscan,dscan,urscan,ulscan,dlscan,drscan)

    for i in range(dist):
        if pygame.sprite.spritecollideany(rscan,spritesgroup):
            if pygame.sprite.spritecollideany(rscan,ogroup):
                results[0] = [1,i]
            elif pygame.sprite.spritecollideany(rscan,ggroup):
                results[0] = [2,i]
            elif pygame.sprite.spritecollideany(rscan,dgroup):
                results[0] = [3,i]
            break
        rscan.update()

    for i in range(dist):
        if pygame.sprite.spritecollideany(uscan,spritesgroup):
            if pygame.sprite.spritecollideany(uscan,ogroup):
                results[1] = [1,i]
            elif pygame.sprite.spritecollideany(uscan,ggroup):
                results[1] = [2,i]
            elif pygame.sprite.spritecollideany(uscan,dgroup):
                results[1] = [3,i]
            break
        uscan.update()

    for i in range(dist):
        if pygame.sprite.spritecollideany(lscan,spritesgroup):
            if pygame.sprite.spritecollideany(lscan,ogroup):
                results[2] = [1,i]
            elif pygame.sprite.spritecollideany(lscan,ggroup):
                results[2] = [2,i]
            elif pygame.sprite.spritecollideany(lscan,dgroup):
                results[2] = [3,i]
            break
        lscan.update()

    for i in range(dist):
        if pygame.sprite.spritecollideany(dscan,spritesgroup):
            if pygame.sprite.spritecollideany(dscan,ogroup):
                results[3] = [1,i]
            elif pygame.sprite.spritecollideany(dscan,ggroup):
                results[3] = [2,i]
            elif pygame.sprite.spritecollideany(dscan,dgroup):
                results[3] = [3,i]
            break
        dscan.update()

    for i in range(dist):
        if pygame.sprite.spritecollideany(urscan,spritesgroup):
            if pygame.sprite.spritecollideany(urscan,ogroup):
                results[4] = [1,i+19]
            elif pygame.sprite.spritecollideany(urscan,ggroup):
                results[4] = [2,i+19]
            elif pygame.sprite.spritecollideany(urscan,dgroup):
                results[4] = [3,i+19]
            break
        urscan.update()

    for i in range(dist):
        if pygame.sprite.spritecollideany(ulscan,spritesgroup):
            if pygame.sprite.spritecollideany(ulscan,ogroup):
                results[5] = [1,i+19]
            elif pygame.sprite.spritecollideany(ulscan,ggroup):
                results[5] = [2,i+19]
            elif pygame.sprite.spritecollideany(ulscan,dgroup):
                results[5] = [3,i+19]
            break
        ulscan.update()

    for i in range(dist):
        if pygame.sprite.spritecollideany(dlscan,spritesgroup):
            if pygame.sprite.spritecollideany(dlscan,ogroup):
                results[6] = [1,i+19]
            elif pygame.sprite.spritecollideany(dlscan,ggroup):
                results[6] = [2,i+19]
            elif pygame.sprite.spritecollideany(dlscan,dgroup):
                results[6] = [3,i+19]
            break
        dlscan.update()

    for i in range(dist):
        if pygame.sprite.spritecollideany(drscan,spritesgroup):
            if pygame.sprite.spritecollideany(drscan,ogroup):
                results[7] = [1,i+19]
            elif pygame.sprite.spritecollideany(drscan,ggroup):
                results[7] = [2,i+19]
            elif pygame.sprite.spritecollideany(drscan,dgroup):
                results[7] = [3,i+19]
            break
        drscan.update()


    for entity in scansgroup:
        entity.kill()

    return results

In [54]:
#ONESIM() DEFINITION

def onesim(maxlen):
    pygame.init()

    clock = pygame.time.Clock()

    #Sets the height and width of the environment
    SCREEN_WIDTH = 800
    SCREEN_HEIGHT = 600

    screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))

    # output array is num_frames*num_drones*num_coords
    posarr = np.zeros((maxlen,6,3))

    allsprites = pygame.sprite.Group()

    # 6 drones are currently hardcoded in with starting positions
    d1 = Drone(1,np.random.randint(50,100),200,0,1)
    d2 = Drone(2,np.random.randint(50,100),250,0,1)
    d3 = Drone(3,np.random.randint(50,100),300,0,1)
    d4 = Drone(4,np.random.randint(50,100),350,0,2)
    d5 = Drone(5,np.random.randint(50,100),400,0,2)
    d6 = Drone(6,np.random.randint(50,100),450,0,2)

    drones = pygame.sprite.Group()
    drones.add(d1,d2,d3,d4,d5,d6)
    allsprites.add(d1,d2,d3,d4,d5,d6)

    dronelist = [d1,d2,d3,d4,d5,d6]

    # A green 'goal' is randomly placed somehwere on the right side of the environment
    ygoal = np.random.randint(150,450)
    endgoal = Goal(750,ygoal,0)

    goals = pygame.sprite.Group()
    goals.add(endgoal)
    allsprites.add(endgoal)

    goal_l = endgoal.rect.left
    goal_t = endgoal.rect.top

    # Two obstacles randomly placed in the middle of the environment
    yo1 = np.random.randint(150,450)
    xo1 = np.random.randint(250,450)
    o1 = Obstacle(xo1,yo1,0,20,300,0)

    yo2 = np.random.randint(150,450)
    xo2 = np.random.randint(450,650)
    o2 = Obstacle(550,yo2,0,20,300,0)

    obstacles = pygame.sprite.Group()
    obstacles.add(o1,o2)
    allsprites.add(o1,o2)

    # If running is ever false, the run terminates
    running = True
    counter = 0

    scans = pygame.sprite.Group()

    while running:
        # Can force exit the simulation using ESC or by closing the window
        for event in pygame.event.get():
            if event.type == KEYDOWN:
                if event.key == K_ESCAPE:
                    running = False

            elif event.type == QUIT:
                running = False

        for entity in drones:
            x = observe(entity,allsprites,100,scans,obstacles,goals,drones)
            #print(x)


        reward = [0,0,0,0,0,0]


        # Drones update, according to the update function defined in Drone class
        #drones.update(SCREEN_WIDTH,SCREEN_HEIGHT)
        for entity in drones:
            prevl = entity.rect.left
            prevt = entity.rect.top
            prevdist = np.sqrt((goal_l-prevl)**2 + (goal_t-prevt)**2)

            entity.update(SCREEN_WIDTH,SCREEN_HEIGHT)
            
            nowl = entity.rect.left
            nowt = entity.rect.top
            nowdist = np.sqrt((goal_l-nowl)**2 + (goal_t-nowt)**2)

            gamma = 0.999

            indireward = gamma**counter * (prevdist-nowdist)

            index=entity.ID - 1

            reward[index] = indireward

            #print(goal_l,goal_t,prevl,prevt,prevdist,nowl,nowt,nowdist,reward)

        #print()



        screen.fill((0,0,0))

        for entity in allsprites:
            screen.blit(entity.surf, entity.rect)
            
        for entity in drones:
            # Drone is killed if it collides with an obstacle
            if pygame.sprite.spritecollideany(entity, obstacles):
                entity.kill()
                reward[entity.ID-1] -= 100

            # Drone is killed if it collides with the goal (success!)
            if pygame.sprite.spritecollideany(entity,goals):
                entity.kill()

            # Both drones are killed if they collide with each other
            for entity2 in drones:
                if entity != entity2 and pygame.sprite.collide_rect(entity, entity2):
                    entity.kill()
                    entity2.kill()
                    reward[entity.ID-1] -= 100
                    reward[entity2.ID-1] -= 100

        print(reward)
    
        pygame.display.flip()

        dead = 0

        # Store results in posarr
        for i in range(len(dronelist)):
            entity = dronelist[i]
            if entity.alive():
                posarr[counter,i] = [entity.rect.left,entity.rect.top,0]
            else:
                dead += 1
                posarr[counter,i] = [-1,-1,-1]

        # Run ends early if all drones are dead
        if dead == 6:
            running = False

        # Run ends if max length has been reached
        counter += 1
        if counter >= maxlen:
            running = False

        # I'm not sure if this is helping the code run more smoothly or not
        clock.tick(60)

    pygame.quit()
    
    return posarr

In [19]:
#MULTISIM() DEFINITION

def multisim(numsims,maxlen):
    multisimlist = []
    for i in range(numsims):
        x=onesim(maxlen)
        multisimlist.append(x)
    multisimlist=np.array(multisimlist)

    return multisimlist

In [55]:
x=onesim(100)

[4.061206837497252, -4.425683862306187, -7.318424710524937, 9.941337594000743, 9.845235228036358, 9.695554267247985]
[3.9174186730045664, -1.6159653261615496, 8.274008772293621, 9.92962572394885, 9.830924432447524, 9.677201938911415]
[-4.8068360776912, 10.159744592858887, 9.856150805854682, 9.917846208408648, 9.816437951384072, 9.658508275431982]
[7.19994171591153, 3.5681178188737097, 9.961332651753654, 9.905994258615364, 9.801764987990657, 9.63945352153189]
[-6.103573273843353, 4.515797237053245, 9.951096298077424, 9.89406471285451, 9.786893956103846, 9.62001653875695]
[3.872849581262055, 7.456842586453359, 9.804007519279578, 9.882052001064677, 9.771812410526223, 9.600174689917072]
[9.905182533964606, -1.8166510050720694, -7.379980423593583, 9.869950105470735, 9.756506970081848, 9.579903712371406]
[-1.9022074031941492, -4.824390268190281, 8.315671618152773, 9.857752516731383, 9.740963232593131, 9.559177578945832]
[-9.969757450579236, 7.122705030492812, 8.237907961479392, 9.84545218500

In [98]:
print(x[99])

[[ 96. 212.   0.]
 [ -1.  -1.  -1.]
 [ -1.  -1.  -1.]
 [ -1.  -1.  -1.]
 [ -1.  -1.  -1.]
 [ -1.  -1.  -1.]]


In [7]:
z=multisim(3,300)

[[ 0  0]
 [ 0  0]
 [ 3 29]
 [ 0  0]]
[[0 0]
 [0 0]
 [0 0]
 [0 0]]
[[0 0]
 [0 0]
 [0 0]
 [0 0]]
[[ 3 29]
 [ 0  0]
 [ 0  0]
 [ 0  0]]
[[0 0]
 [0 0]
 [0 0]
 [0 0]]
[[0 0]
 [0 0]
 [0 0]
 [0 0]]
[[ 0  0]
 [ 0  0]
 [ 3 26]
 [ 3 97]]
[[0 0]
 [0 0]
 [0 0]
 [0 0]]
[[ 0  0]
 [ 0  0]
 [ 0  0]
 [ 3 35]]
[[ 3 26]
 [ 3 35]
 [ 0  0]
 [ 0  0]]
[[ 0  0]
 [ 3 97]
 [ 0  0]
 [ 0  0]]
[[0 0]
 [0 0]
 [0 0]
 [0 0]]
[[0 0]
 [0 0]
 [0 0]
 [0 0]]
[[0 0]
 [0 0]
 [0 0]
 [0 0]]
[[0 0]
 [0 0]
 [0 0]
 [0 0]]
[[0 0]
 [0 0]
 [0 0]
 [0 0]]
[[0 0]
 [0 0]
 [0 0]
 [0 0]]
[[0 0]
 [0 0]
 [0 0]
 [0 0]]
[[ 0  0]
 [ 0  0]
 [ 3 20]
 [ 3 93]]
[[0 0]
 [0 0]
 [0 0]
 [0 0]]
[[0 0]
 [0 0]
 [0 0]
 [0 0]]
[[ 3 20]
 [ 0  0]
 [ 0  0]
 [ 0  0]]
[[ 0  0]
 [ 3 93]
 [ 0  0]
 [ 0  0]]
[[0 0]
 [0 0]
 [0 0]
 [0 0]]
[[0 0]
 [0 0]
 [3 6]
 [0 0]]
[[0 0]
 [0 0]
 [0 0]
 [0 0]]
[[0 0]
 [0 0]
 [0 0]
 [0 0]]
[[3 6]
 [0 0]
 [0 0]
 [0 0]]
[[0 0]
 [0 0]
 [0 0]
 [0 0]]
[[0 0]
 [0 0]
 [0 0]
 [0 0]]
[[0 0]
 [0 0]
 [0 0]
 [0 0]]
[[0 0]
 [0 0]
 [0 0]
 [0 0]]


In [40]:
np.shape(z)

(3, 100, 6, 3)

In [63]:
def potential(state,goal_l,goal_t):
    p = 0
    for drone in state:
        p += abs(drone[0]-goal_l)
        p += abs(drone[1]-goal_t)
    return p

In [168]:
#Single drone - code still regularly iterates through the drone list because it will help later

pygame.init()

clock = pygame.time.Clock()

#Sets the height and width of the environment
SCREEN_WIDTH = 800
SCREEN_HEIGHT = 600

screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))

# output array is num_frames*num_drones*num_coords
# posarr = np.zeros((maxlen,6,3))

allsprites = pygame.sprite.Group()

# 6 drones are currently hardcoded in with starting positions
d1 = Drone_NSEW(1,np.random.randint(50,100),300,0)

dinits = [[d1.rect.left,d1.rect.top]]

drones = pygame.sprite.Group()
drones.add(d1)
allsprites.add(d1)

for entity in drones:
    entity.force = 10

dronelist = [d1]

# A green 'goal' is randomly placed somehwere on the right side of the environment
ygoal = np.random.randint(150,450)
endgoal = Goal(750,ygoal,0)

goals = pygame.sprite.Group()
goals.add(endgoal)
allsprites.add(endgoal)

goal_l = endgoal.rect.left
goal_t = endgoal.rect.top

# Two obstacles randomly placed in the middle of the environment
yo1 = np.random.randint(150,450)
xo1 = np.random.randint(250,450)
o1 = Obstacle(xo1,yo1,0,20,300,0)

yo2 = np.random.randint(150,450)
xo2 = np.random.randint(450,650)
o2 = Obstacle(550,yo2,0,20,300,0)

oinits =((o1.rect.left,o1.rect.top),(o2.rect.left,o2.rect.top))

obstacles = pygame.sprite.Group()
obstacles.add(o1,o2)
allsprites.add(o1,o2)

# If running is ever false, the run terminates
running = True

states = {}
sources = {}

states[0] = dinits
sources[0] = -1

currentstate = dinits
currentdist = 0
currentID = 0


currentpotential = potential(dinits,goal_l,goal_t)
priorityqueue = [0]
distlist = [0]
potlist = [currentpotential]
astarlist = [currentpotential]

donequeue = []
donedistlist = []
donepotlist = []
doneastarlist = []

counter = 1

#screen.fill((0,0,0))
#for entity in allsprites:
#    screen.blit(entity.surf, entity.rect)
#pygame.display.flip()
#pygame.time.wait(10000)

while running:
    # Can force exit the simulation using ESC or by closing the window
    for event in pygame.event.get():
        if event.type == KEYDOWN:
            if event.key == K_ESCAPE:
                running = False

        elif event.type == QUIT:
            running = False

    

    commands = [0]

    for a in ['u','d','r','l']:
        commands[0] = a

        for i in range(len(dronelist)):
            entity = dronelist[i]
            entity.rect.left = currentstate[i][0]
            entity.rect.top = currentstate[i][1]
            
        for i in range(len(dronelist)):
            entity = dronelist[i]
            entity.move = commands[i]
            entity.update(SCREEN_WIDTH, SCREEN_HEIGHT)
                            
        crash = 0
        for entity in drones:
            if pygame.sprite.spritecollideany(entity,obstacles):
                crash = 1
            else:
                for entity2 in drones:
                    if entity != entity2 and pygame.sprite.collide_rect(entity, entity2):
                        crash = 1
                            
                        

        dronepos = [[d1.rect.left,d1.rect.top]]

        if crash == 0 and dronepos in states.values():
            ID = [k for k, v in states.items() if v == dronepos]
            if len(ID) > 1:
                print('You are about to encounter some never before seen errors')
            ID = ID[0]
            if ID in priorityqueue and ID in donequeue:
                print('Not good')


            elif ID in priorityqueue:
                pos = priorityqueue.index(ID)
                newdist = currentdist + 10
                if newdist < distlist[pos]:
                    x = priorityqueue.pop(pos)
                    y = distlist.pop(pos)
                    y = currentdist + 10
                    z = potlist.pop(pos)
                    w = astarlist.pop(pos)
                    w = y + z

                    sources[ID] = currentID

                    if w >= astarlist[-1]:
                        priorityqueue.append(x)
                        distlist.append(y)
                        potlist.append(z)
                        astarlist.append(w)
                    else:
                        for i in range(1,len(astarlist)):
                            if w <= astarlist[i]:
                                index=i
                                priorityqueue.insert(index,x)
                                distlist.insert(index,y)
                                potlist.insert(index,z)
                                astarlist.insert(index,w)
                                break



            elif ID in donequeue:
                pos = donequeue.index(ID)
                newdist = currentdist + 10
                if newdist < donedistlist[pos]:
                    x = donequeue.pop(pos)
                    y = donedistlist.pop(pos)
                    y = currentdist + 10
                    z = donepotlist.pop(pos)
                    w = doneastarlist.pop(pos)
                    w = y + z

                    sources[ID] = currentID

                    if w >= astarlist[-1]:
                        priorityqueue.append(x)
                        distlist.append(y)
                        potlist.append(z)
                        astarlist.append(w)
                    else:
                        for i in range(1,len(astarlist)):
                            if w <= astarlist[i]:
                                index=i
                                priorityqueue.insert(index,x)
                                distlist.insert(index,y)
                                potlist.insert(index,z)
                                astarlist.insert(index,w)
                                break


            else:
                print('Not good')
            

        if crash == 0 and dronepos not in states.values():
            #print('hell yeah')
            states[counter] = dronepos
            sources[counter] = currentID
            dist = currentdist + 10
            pot = potential(dronepos,goal_l,goal_t)
            astar = dist+pot
            #print(astar)
            if astar >= astarlist[-1]:
                priorityqueue.append(counter)
                distlist.append(dist)
                potlist.append(pot)
                astarlist.append(astar)
            else:
                for i in range(1,len(astarlist)):
                    if astar <= astarlist[i]:
                        index=i
                        priorityqueue.insert(index,counter)
                        distlist.insert(index,dist)
                        potlist.insert(index,pot)
                        astarlist.insert(index,astar)
                        break

            counter += 1
            #print(counter)


        for i in range(len(dronelist)):
            entity = dronelist[i]
            entity.rect.left = currentstate[i][0]
            entity.rect.top = currentstate[i][1]



    screen.fill((0,0,0))

    for entity in allsprites:
        screen.blit(entity.surf, entity.rect)
    
    pygame.display.flip()

    x = priorityqueue.pop(0)
    y = distlist.pop(0)
    z = potlist.pop(0)
    w = astarlist.pop(0)

    donequeue.append(x)
    donedistlist.append(y)
    donepotlist.append(z)
    doneastarlist.append(w)

    currentID = priorityqueue[0]
    currentstate = states[currentID]
    currentdist = distlist[0]
    currentpotential = potlist[0]

    if currentpotential <= 10:
        running=False

    if counter >= 100000:
        running=False

pygame.quit()
    
print(dinits)
print(goal_l,goal_t)
print(oinits)
#print(len(priorityqueue))
#print(donequeue)

[[46, 295]]
745 160
((286, 133), (540, 71))


In [169]:
shortestpath = []
nowstate = priorityqueue[0]
shortestpath.append(nowstate)
while True:
    prevstate = sources[nowstate]
    if prevstate == -1:
        break
    else:
        nowstate = prevstate
        shortestpath.append(nowstate)

print(shortestpath)

[2529, 2525, 2522, 2519, 2516, 2513, 2510, 2507, 2504, 2502, 2500, 2498, 2495, 2492, 2489, 2486, 2483, 2480, 2477, 2474, 2471, 2468, 2465, 2462, 2459, 2456, 2453, 2450, 2447, 2445, 2443, 2441, 2231, 1904, 1744, 1584, 1424, 766, 765, 762, 759, 756, 753, 750, 747, 744, 741, 738, 735, 732, 729, 726, 723, 720, 717, 714, 711, 708, 705, 702, 699, 697, 695, 623, 554, 487, 422, 96, 94, 92, 90, 88, 86, 84, 82, 80, 78, 76, 74, 73, 71, 70, 67, 64, 61, 58, 55, 52, 49, 46, 43, 40, 37, 34, 31, 28, 25, 22, 19, 16, 13, 10, 7, 3, 0]


In [171]:
pygame.init()

clock = pygame.time.Clock()

#Sets the height and width of the environment
SCREEN_WIDTH = 800
SCREEN_HEIGHT = 600

screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))

# output array is num_frames*num_drones*num_coords
# posarr = np.zeros((maxlen,6,3))

allsprites = pygame.sprite.Group()

d1 = Drone_NSEW(1,np.random.randint(50,100),300,0)

d1.rect.left = dinits[0][0]
d1.rect.top = dinits[0][1]

drones = pygame.sprite.Group()
drones.add(d1)
allsprites.add(d1)

dronelist = [d1]

# A green 'goal' is randomly placed somehwere on the right side of the environment
ygoal = np.random.randint(150,450)
endgoal = Goal(750,ygoal,0)
endgoal.rect.left = goal_l
endgoal.rect.top = goal_t

goals = pygame.sprite.Group()
goals.add(endgoal)
allsprites.add(endgoal)

# Two obstacles randomly placed in the middle of the environment
yo1 = np.random.randint(150,450)
xo1 = np.random.randint(250,450)
o1 = Obstacle(xo1,yo1,0,20,300,0)
o1.rect.left = oinits[0][0]
o1.rect.top = oinits[0][1]

yo2 = np.random.randint(150,450)
xo2 = np.random.randint(450,650)
o2 = Obstacle(550,yo2,0,20,300,0)
o2.rect.left = oinits[1][0]
o2.rect.top = oinits[1][1]

obstacles = pygame.sprite.Group()
obstacles.add(o1,o2)
allsprites.add(o1,o2)

# If running is ever false, the run terminates

screen.fill((0,0,0))
for entity in allsprites:
    screen.blit(entity.surf, entity.rect)
pygame.display.flip()
pygame.time.wait(10000)

for i in range(len(shortestpath)):

    thisstate = shortestpath[-1*(i+1)]
    dronepos = states[thisstate]

    for i in range(len(dronelist)):
        entity = dronelist[i]
        entity.rect.left = dronepos[i][0]
        entity.rect.top = dronepos[i][1]

    screen.fill((0,0,0))

    for entity in allsprites:
        screen.blit(entity.surf, entity.rect)
    
    pygame.display.flip()

pygame.quit()

In [ ]:
stop going past this block you doofus

In [167]:
def goodformation(dronepos):
    penalty = 0
    for i in range(len(dronepos)):
        for j in range(i,len(dronepos)):
            dist = abs(dronepos[i][0]-dronepos[j][0]) + abs(dronepos[i][1]-dronepos[j][1])
            if dist >= 50:
                penalty += dist

    return penalty

In [166]:
#Single drone - code still regularly iterates through the drone list because it will help later

pygame.init()

clock = pygame.time.Clock()

#Sets the height and width of the environment
SCREEN_WIDTH = 800
SCREEN_HEIGHT = 600

screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))

# output array is num_frames*num_drones*num_coords
# posarr = np.zeros((maxlen,6,3))

allsprites = pygame.sprite.Group()

# 6 drones are currently hardcoded in with starting positions
d1 = Drone_NSEW(1,np.random.randint(50,100),275,0)
d2 = Drone_NSEW(1,np.random.randint(50,100),325,0)

dinits = [[d1.rect.left,d1.rect.top],[d2.rect.left,d2.rect.top]]

drones = pygame.sprite.Group()
drones.add(d1,d2)
allsprites.add(d1,d2)

for entity in drones:
    entity.force = 10

dronelist = [d1,d2]

# A green 'goal' is randomly placed somehwere on the right side of the environment
ygoal = np.random.randint(150,450)
endgoal = Goal(750,ygoal,0)

goals = pygame.sprite.Group()
goals.add(endgoal)
allsprites.add(endgoal)

goal_l = endgoal.rect.left
goal_t = endgoal.rect.top

# Two obstacles randomly placed in the middle of the environment
yo1 = np.random.randint(150,450)
xo1 = np.random.randint(250,450)
o1 = Obstacle(xo1,yo1,0,20,300,0)

yo2 = np.random.randint(150,450)
xo2 = np.random.randint(450,650)
o2 = Obstacle(550,yo2,0,20,300,0)

oinits =((o1.rect.left,o1.rect.top),(o2.rect.left,o2.rect.top))

obstacles = pygame.sprite.Group()
obstacles.add(o1,o2)
allsprites.add(o1,o2)

# If running is ever false, the run terminates
running = True

states = {}
sources = {}

states[0] = dinits
sources[0] = -1

currentstate = dinits
currentdist = 0
currentID = 0


currentpotential = potential(dinits,goal_l,goal_t)
priorityqueue = [0]
distlist = [0]
potlist = [currentpotential]
astarlist = [currentpotential]

donequeue = []
donedistlist = []
donepotlist = []
doneastarlist = []

counter = 1

screen.fill((0,0,0))
for entity in allsprites:
    screen.blit(entity.surf, entity.rect)
pygame.display.flip()
pygame.time.wait(10000)

while running:
    # Can force exit the simulation using ESC or by closing the window
    for event in pygame.event.get():
        if event.type == KEYDOWN:
            if event.key == K_ESCAPE:
                running = False

        elif event.type == QUIT:
            running = False

    

    commands = [0,0]

    for a in ['u','d','r','l']:
        commands[0] = a
        for b in ['u','d','r','l']:
            commands[1] = b


            for i in range(len(dronelist)):
                entity = dronelist[i]
                entity.rect.left = currentstate[i][0]
                entity.rect.top = currentstate[i][1]
            
            for i in range(len(dronelist)):
                entity = dronelist[i]
                entity.move = commands[i]
                entity.update(SCREEN_WIDTH, SCREEN_HEIGHT)
                            
            crash = 0
            for entity in drones:
                if pygame.sprite.spritecollideany(entity,obstacles):
                    crash = 1
                else:
                    for entity2 in drones:
                        if entity != entity2 and pygame.sprite.collide_rect(entity, entity2):
                            crash = 1
                            
                        

            dronepos = [[d1.rect.left,d1.rect.top],[d2.rect.left,d2.rect.top]]

            penalty = goodformation(dronepos)

            """if crash == 0 and dronepos in states.values():
                ID = [k for k, v in states.items() if v == dronepos]
                if len(ID) > 1:
                    print('You are about to encounter some never before seen errors')
                ID = ID[0]
                if ID in priorityqueue and ID in donequeue:
                    print('Not good')


                elif ID in priorityqueue:
                    pos = priorityqueue.index(ID)
                    newdist = currentdist + 1
                    if newdist < distlist[pos]:
                        x = priorityqueue.pop(pos)
                        y = distlist.pop(pos)
                        y = currentdist + 1
                        z = potlist.pop(pos)
                        w = astarlist.pop(pos)
                        w = y + z

                        sources[ID] = currentID

                        if w >= astarlist[-1]:
                            priorityqueue.append(x)
                            distlist.append(y)
                            potlist.append(z)
                            astarlist.append(w)
                        else:
                            for i in range(1,len(astarlist)):
                                if w <= astarlist[i]:
                                    index=i
                                    priorityqueue.insert(index,x)
                                    distlist.insert(index,y)
                                    potlist.insert(index,z)
                                    astarlist.insert(index,w)
                                    break



                elif ID in donequeue:
                    pos = donequeue.index(ID)
                    newdist = currentdist + 1
                    if newdist < donedistlist[pos]:
                        x = donequeue.pop(pos)
                        y = donedistlist.pop(pos)
                        y = currentdist + 1
                        z = donepotlist.pop(pos)
                        w = doneastarlist.pop(pos)
                        w = y + z

                        sources[ID] = currentID

                        if w >= astarlist[-1]:
                            priorityqueue.append(x)
                            distlist.append(y)
                            potlist.append(z)
                            astarlist.append(w)
                        else:
                            for i in range(1,len(astarlist)):
                                if w <= astarlist[i]:
                                    index=i
                                    priorityqueue.insert(index,x)
                                    distlist.insert(index,y)
                                    potlist.insert(index,z)
                                    astarlist.insert(index,w)
                                    break


                else:
                    print('Not good')"""
            

            if crash == 0 and dronepos not in states.values():
                #print('hell yeah')
                states[counter] = dronepos
                sources[counter] = currentID
                dist = currentdist + 10
                pot = potential(dronepos,goal_l,goal_t)
                astar = dist+pot
                #print(astar)
                if astar >= astarlist[-1]:
                    priorityqueue.append(counter)
                    distlist.append(dist)
                    potlist.append(pot)
                    astarlist.append(astar)
                else:
                    for i in range(1,len(astarlist)):
                        if astar <= astarlist[i]:
                            index=i
                            priorityqueue.insert(index,counter)
                            distlist.insert(index,dist)
                            potlist.insert(index,pot)
                            astarlist.insert(index,astar)
                            break

                counter += 1
                #print(counter)

            for i in range(len(dronelist)):
                entity = dronelist[i]
                entity.rect.left = currentstate[i][0]
                entity.rect.top = currentstate[i][1]



    screen.fill((0,0,0))

    for entity in allsprites:
        screen.blit(entity.surf, entity.rect)
    
    pygame.display.flip()

    x = priorityqueue.pop(0)
    y = distlist.pop(0)
    z = potlist.pop(0)
    w = astarlist.pop(0)

    donequeue.append(x)
    donedistlist.append(y)
    donepotlist.append(z)
    doneastarlist.append(w)

    currentID = priorityqueue[0]
    currentstate = states[currentID]
    currentdist = distlist[0]
    currentpotential = potlist[0]

    if currentpotential <= 50:
        running=False

    #if counter >= 100000:
    #    running=False

pygame.quit()
    
print(dinits)
print(goal_l,goal_t)
print(oinits)
#print(len(priorityqueue))
#print(donequeue)

[[54, 270], [47, 320]]
745 401
((346, 274), (540, 53))


In [163]:
shortestpath = []
nowstate = priorityqueue[0]
shortestpath.append(nowstate)
while True:
    prevstate = sources[nowstate]
    if prevstate == -1:
        break
    else:
        nowstate = prevstate
        shortestpath.append(nowstate)

print(shortestpath)

[1379, 1373, 1355, 1349, 1331, 1325, 1307, 1301, 1283, 1277, 1259, 1253, 1235, 1229, 1211, 1205, 1187, 1181, 1163, 1157, 1139, 1133, 1115, 1109, 1091, 1085, 1070, 1066, 1046, 1041, 1026, 1022, 1008, 976, 951, 942, 931, 924, 913, 906, 895, 888, 877, 870, 859, 852, 841, 834, 823, 816, 805, 798, 787, 780, 769, 762, 751, 744, 733, 726, 715, 708, 700, 696, 682, 675, 667, 663, 655, 651, 642, 627, 622, 616, 606, 599, 581, 572, 555, 540, 525, 510, 495, 480, 465, 450, 435, 420, 405, 390, 375, 360, 345, 330, 315, 300, 285, 270, 255, 240, 225, 211, 196, 192, 177, 162, 147, 133, 121, 106, 91, 76, 61, 46, 31, 16, 0]


In [164]:
pygame.init()

clock = pygame.time.Clock()

#Sets the height and width of the environment
SCREEN_WIDTH = 800
SCREEN_HEIGHT = 600

screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))

# output array is num_frames*num_drones*num_coords
# posarr = np.zeros((maxlen,6,3))

allsprites = pygame.sprite.Group()

d1 = Drone_NSEW(1,np.random.randint(50,100),300,0)
d2 = Drone_NSEW(2,np.random.randint(50,100),250,0)

d1.rect.left = dinits[0][0]
d1.rect.top = dinits[0][1]
d2.rect.left = dinits[1][0]
d2.rect.top = dinits[1][1]

drones = pygame.sprite.Group()
drones.add(d1,d2)
allsprites.add(d1,d2)

dronelist = [d1,d2]

# A green 'goal' is randomly placed somehwere on the right side of the environment
ygoal = np.random.randint(150,450)
endgoal = Goal(750,ygoal,0)
endgoal.rect.left = goal_l
endgoal.rect.top = goal_t

goals = pygame.sprite.Group()
goals.add(endgoal)
allsprites.add(endgoal)

# Two obstacles randomly placed in the middle of the environment
yo1 = np.random.randint(150,450)
xo1 = np.random.randint(250,450)
o1 = Obstacle(xo1,yo1,0,20,300,0)
o1.rect.left = oinits[0][0]
o1.rect.top = oinits[0][1]

yo2 = np.random.randint(150,450)
xo2 = np.random.randint(450,650)
o2 = Obstacle(550,yo2,0,20,300,0)
o2.rect.left = oinits[1][0]
o2.rect.top = oinits[1][1]

obstacles = pygame.sprite.Group()
obstacles.add(o1,o2)
allsprites.add(o1,o2)

# If running is ever false, the run terminates

screen.fill((0,0,0))
for entity in allsprites:
    screen.blit(entity.surf, entity.rect)
pygame.display.flip()
pygame.time.wait(10000)

for i in range(len(shortestpath)):

    thisstate = shortestpath[-1*(i+1)]
    dronepos = states[thisstate]

    for i in range(len(dronelist)):
        entity = dronelist[i]
        entity.rect.left = dronepos[i][0]
        entity.rect.top = dronepos[i][1]

    screen.fill((0,0,0))

    for entity in allsprites:
        screen.blit(entity.surf, entity.rect)
    
    pygame.display.flip()

pygame.quit()

In [ ]:
not here either

In [106]:
pygame.init()

clock = pygame.time.Clock()

#Sets the height and width of the environment
SCREEN_WIDTH = 800
SCREEN_HEIGHT = 600

screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))

# output array is num_frames*num_drones*num_coords
# posarr = np.zeros((maxlen,6,3))

allsprites = pygame.sprite.Group()

# 6 drones are currently hardcoded in with starting positions
d1 = Drone_NSEW(1,np.random.randint(50,100),200,0)
d2 = Drone_NSEW(2,np.random.randint(50,100),250,0)
d3 = Drone_NSEW(3,np.random.randint(50,100),300,0)
d4 = Drone_NSEW(4,np.random.randint(50,100),350,0)
d5 = Drone_NSEW(5,np.random.randint(50,100),400,0)
d6 = Drone_NSEW(6,np.random.randint(50,100),450,0)

dinits = [[d1.rect.left,d1.rect.top], [d2.rect.left,d2.rect.top], [d3.rect.left,d3.rect.top], [d4.rect.left,d4.rect.top], [d5.rect.left,d5.rect.top], [d6.rect.left,d6.rect.top]]

drones = pygame.sprite.Group()
drones.add(d1,d2,d3,d4,d5,d6)
allsprites.add(d1,d2,d3,d4,d5,d6)

for entity in drones:
    entity.force = 1

dronelist = [d1,d2,d3,d4,d5,d6]

# A green 'goal' is randomly placed somehwere on the right side of the environment
ygoal = np.random.randint(150,450)
endgoal = Goal(750,ygoal,0)

goals = pygame.sprite.Group()
goals.add(endgoal)
allsprites.add(endgoal)

goal_l = endgoal.rect.left
goal_t = endgoal.rect.top

# Two obstacles randomly placed in the middle of the environment
yo1 = np.random.randint(150,450)
xo1 = np.random.randint(250,450)
o1 = Obstacle(xo1,yo1,0,20,300,0)

yo2 = np.random.randint(150,450)
xo2 = np.random.randint(450,650)
o2 = Obstacle(550,yo2,0,20,300,0)

oinits =((o1.rect.left,o1.rect.top),(o2.rect.left,o2.rect.top))

obstacles = pygame.sprite.Group()
obstacles.add(o1,o2)
allsprites.add(o1,o2)

# If running is ever false, the run terminates
running = True

states = {}
sources = {}

states[0] = dinits
sources[0] = 'start'

currentstate = dinits
currentdist = 0
currentID = 0


initpotential = potential(dinits,goal_l,goal_t)
priorityqueue = [0]
distlist = [0]
potlist = [initpotential]
astarlist = [initpotential]

donequeue = []
donedistlist = []
donepotlist = []
doneastarlist = []

counter = 1



while running:
    # Can force exit the simulation using ESC or by closing the window
    for event in pygame.event.get():
        if event.type == KEYDOWN:
            if event.key == K_ESCAPE:
                running = False

        elif event.type == QUIT:
            running = False

    

    commands = [0,0,0,0,0,0]

    for a in ['r','u','l','d']:
        commands[0] = a
        for b in ['r','u','l','d']:
            commands[1] = b
            for c in ['r','u','l','d']:
                commands[2]=c
                for d in ['r','u','l','d']:
                    commands[3]=d
                    for e in ['r','u','l','d']:
                        commands[4] = e
                        for f in ['r','u','l','d']:
                            commands[5] = f
                            for i in range(len(dronelist)):
                                entity = dronelist[i]
                                entity.move = commands[i]
                                entity.update(SCREEN_WIDTH, SCREEN_HEIGHT)
                            
                            crash = 0
                            for entity in drones:
                                if pygame.sprite.spritecollideany(entity,obstacles):
                                    crash = 1
                                else:
                                    for entity2 in drones:
                                        if entity != entity2 and pygame.sprite.collide_rect(entity, entity2):
                                            crash = 1
                            
                        

                            dronepos = [[d1.rect.left,d1.rect.top], [d2.rect.left,d2.rect.top], [d3.rect.left,d3.rect.top], [d4.rect.left,d4.rect.top], [d5.rect.left,d5.rect.top], [d6.rect.left,d6.rect.top]]
            

                            if crash == 0 and dronepos not in states.values():
                                #print('hell yeah')
                                states[counter] = dronepos
                                sources[counter] = currentID
                                dist = currentdist + 1
                                pot = potential(dronepos,goal_l,goal_t)
                                astar = dist+pot
                                #print(astar)
                                if astar >= astarlist[-1]:
                                    priorityqueue.append(counter)
                                    distlist.append(dist)
                                    potlist.append(pot)
                                    astarlist.append(astar)
                                else:
                                    for i in range(1,len(astarlist)):
                                        if astar <= astarlist[i]:
                                            index=i
                                            priorityqueue.insert(index,counter)
                                            distlist.insert(index,dist)
                                            potlist.insert(index,pot)
                                            astarlist.insert(index,astar)
                                            break

                                counter += 1
                                #print(counter)

                            for i in range(len(dronelist)):
                                entity = dronelist[i]
                                entity.rect.left = currentstate[i][0]
                                entity.rect.top = currentstate[i][1]



    screen.fill((0,0,0))

    for entity in allsprites:
        screen.blit(entity.surf, entity.rect)
    
    pygame.display.flip()

    x = priorityqueue.pop(0)
    y = distlist.pop(0)
    z = potlist.pop(0)
    w = astar

    running=False

pygame.quit()
    
#print(dinits)
#print(goal_l,goal_t)
#print(oinits)
#print(len(priorityqueue))
#print(astarlist)
#print(donequeue)

In [61]:
qwerty = {}
qwerty[0] = (1,2,3)
if (1,2,3) in qwerty.values():
    print('yay')

yay


In [57]:
print(qwerty)

{0: (1, 2, 3)}
